In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import polars as pl

from datetime import datetime
import time

import utils.source

from utils.request_market_data import (
    request_prices_to_landing,
    write_prices_to_raw,
    trasnform_trusted_prices
)

from utils.request_cvm_ciasabertas import (
    download_files,
    read_files_from_landing,
    write_files_to_lake,
    read_files_from_lake,
    trasnform_trusted_accounts,
    trasnform_trusted_tickers,
    transform_trusted_stocks,
    trasnform_refined_accounts
)

from utils.indicators_consolidation import (
    consolidate_accounts
)


In [66]:
accounts = (
    consolidate_accounts()
    .with_columns_seq(
        QUARTER = pl.col('DT_REFER').dt.quarter(),
        YEAR = pl.col('DT_REFER').dt.year() 
    )
)

In [68]:
stocks = (
    read_files_from_lake(table='fre/capital_social', zone='trusted')
    .with_columns_seq(
        QUARTER = pl.col('DT_REFER').dt.quarter(),
        YEAR = pl.col('DT_REFER').dt.year() 
    )
    .select(
        pl.col('QUARTER'),
        pl.col('YEAR'),
        pl.col('CNPJ'),
        pl.col('TOTAL_STOCKS')
    )
)


In [113]:
x = (
    accounts
    .join(
        stocks,
        on = ['CNPJ', 'QUARTER', 'YEAR'],
        how = 'left'
    )
    .sort('DT_REFER', 'CNPJ')
    .with_columns(
        pl.col('TOTAL_STOCKS').fill_null(strategy="forward").over(['CNPJ'])
    )
    .with_columns(
        pl.col('TOTAL_STOCKS').fill_null(strategy="backward").over(['CNPJ'])
    )
)

In [51]:
stocks.schema

OrderedDict([('CNPJ', Utf8),
             ('DT_REFER', Date),
             ('ON_STOCKS', Int64),
             ('PN_STOCKS', Int64),
             ('TOTAL_STOCKS', Int64)])

In [38]:
active_companies = (
    read_files_from_lake(table='active_companies', zone='refined')
    .select(
        pl.col('CNPJ'),
        pl.col('STOCK'),
        pl.col('SEGMENTO')
    )
)


In [43]:
prices_t = (
    read_files_from_lake(table='prices', zone='trusted')
    .with_columns(
        QUARTER = pl.col('date').dt.quarter()
    )
    .with_columns(
        YEAR = pl.when(pl.col('QUARTER')==4)
            .then(pl.col('date').dt.year()+1)
            .otherwise(pl.col('date').dt.year())  
    )
    .with_columns(
        QUARTER = pl.when(pl.col('QUARTER')==4)
            .then(pl.lit(1))
            .otherwise(pl.col('QUARTER')+1) 
    )
    .group_by(
        ['STOCK', 'QUARTER', 'YEAR']
    )
    .agg(
        pl.col("adjusted_close").median().alias("PRICE")
    )
)

prices_t.schema

OrderedDict([('STOCK', Utf8),
             ('QUARTER', UInt32),
             ('YEAR', Int32),
             ('PRICE', Float64)])

In [46]:
df_consolidated = (
    active_companies
    .join(
        df,
        on = ['CNPJ'],
        how = 'inner'
    )
    .join(
        prices_t,
        on = ['STOCK', 'YEAR', 'QUARTER'],
        how = 'inner'
    )
)

In [47]:
df_consolidated.schema

OrderedDict([('CNPJ', Utf8),
             ('STOCK', Utf8),
             ('SEGMENTO', Utf8),
             ('DT_REFER', Date),
             ('CD_CVM', Int64),
             ('RECEITA', Float64),
             ('LUCRO', Float64),
             ('ATIVO_TOTAL', Float64),
             ('ATIVO_CIRCULANTE', Float64),
             ('PASSIVO_TOTAL', Float64),
             ('PASSIVO_CIRCULANTE', Float64),
             ('PATRIMONIO_LIQUIDO', Float64),
             ('QUARTER', UInt32),
             ('YEAR', Int32),
             ('PRICE', Float64)])

In [23]:
prices_t

STOCK,QUARTER,YEAR,PRICE
str,u32,i32,f64
"""SEDU3""",1,2019,22.05
"""OGXP3""",2,2014,19.0
"""OGXP3""",3,2011,1175.0
"""ENAT3""",4,2021,12.5239
"""ENAT3""",1,2012,8.0251
"""ENAT3""",4,2011,8.8835
"""PETR3""",2,2022,18.4067
"""PETR3""",1,2015,3.477
"""PETR3""",2,2010,11.25725
